<a href="https://colab.research.google.com/github/interactive-fiction-class/interactive-fiction-class.github.io/blob/master/hw4_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning a Pre-trained Transformer for Text Generation

#### Install HuggingFace Transformers library.

In [9]:
!git clone https://github.com/huggingface/transformers

import os
os.chdir('/content/transformers')

!pip install .
!pip install -r ./examples/requirements.txt

os.chdir('/content/transformers/examples')


Cloning into 'transformers'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 19737 (delta 32), reused 34 (delta 13), pack-reused 19671
Receiving objects: 100% (19737/19737), 11.64 MiB | 10.44 MiB/s, done.
Resolving deltas: 100% (14432/14432), done.
Processing /content/transformers
  Created wheel for transformers: filename=transformers-2.5.0-cp36-none-any.whl size=481191 sha256=1352abfe237fc42babdec660e83b7597e8830da0efa0de66facbbb73a416ab7f
  Stored in directory: /tmp/pip-ephem-wheel-cache-d5p0vjgp/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 2.5.0
    Uninstalling transformers-2.5.0:
      Successfully uninstalled transformers-2.5.0


### Finetune a language model on president speeches.

In [12]:
# Download the train and test set.
!wget -nc -O /content/presidential_speeches_test.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_test.txt
!wget -nc -O /content/presidential_speeches_train.txt https://raw.githubusercontent.com/interactive-fiction-class/interactive-fiction-class.github.io/master/homeworks/language-model/presidential_speeches_train.txt

File ‘/content/presidential_speeches_test.txt’ already there; not retrieving.
File ‘/content/presidential_speeches_train.txt’ already there; not retrieving.


Explanation of some of the arguments you might want to modify:

* `line_by_line`: Add `--line_by_line` if distinct lines of the text should be treated as distinct training examples. For example, if your dataset contains one story/tweet/article per line, this should be set.
* `num_train_epochs`: The number of times to iterate over the train set. Increasing the number of epochs may result in better performance, but making this number too high will cause the model to overfit on the train set.
* `block_size` Your training text is truncated into blocks of this length. This is also the maximum length of text sequences you'll be able to do a good job of generating at test time. The total model size is a function of the `block_size` and the `batch_size`. If you are getting out-of-memory errors, then try drecreasing this value.


In [0]:
!python run_language_modeling.py \
    --output_dir=/content/finetuned_model \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --train_data_file=/content/presidential_speeches_train.txt \
    --do_eval \
    --eval_data_file=/content/presidential_speeches_test.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=128 \
    --gradient_accumulation_steps=5

Streaming output truncated to the last 5000 lines.
Iteration:  72% 11426/15836 [1:15:50<28:17,  2.60it/s]
Iteration:  72% 11427/15836 [1:15:50<27:01,  2.72it/s]
Iteration:  72% 11428/15836 [1:15:51<26:02,  2.82it/s]
Iteration:  72% 11429/15836 [1:15:51<25:15,  2.91it/s]
Iteration:  72% 11430/15836 [1:15:52<29:43,  2.47it/s]
Iteration:  72% 11431/15836 [1:15:52<28:18,  2.59it/s]
Iteration:  72% 11432/15836 [1:15:52<26:54,  2.73it/s]
Iteration:  72% 11433/15836 [1:15:53<26:08,  2.81it/s]
Iteration:  72% 11434/15836 [1:15:53<25:32,  2.87it/s]
Iteration:  72% 11435/15836 [1:15:54<29:55,  2.45it/s]
Iteration:  72% 11436/15836 [1:15:54<28:38,  2.56it/s]
Iteration:  72% 11437/15836 [1:15:54<27:35,  2.66it/s]
Iteration:  72% 11438/15836 [1:15:55<26:30,  2.77it/s]
Iteration:  72% 11439/15836 [1:15:55<25:41,  2.85it/s]
Iteration:  72% 11440/15836 [1:15:55<30:38,  2.39it/s]
Iteration:  72% 11441/15836 [1:15:56<29:08,  2.51it/s]
Iteration:  72% 11442/15836 [1:15:56<27:47,  2.63it/s]
Iteration:  72

In [26]:
!python run_generation.py \
--model_type=gpt2 \
--model_name_or_path=/content/model/ \
--k=50 \
--length=128 \
--prompt='<title="Remarks on Mission to Mars>"'

02/20/2020 03:46:33 - INFO - transformers.tokenization_utils -   Model name '/content/model/' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming '/content/model/' is a path, a model identifier, or url to a directory containing tokenizer files.
02/20/2020 03:46:33 - INFO - transformers.tokenization_utils -   Didn't find file /content/model/added_tokens.json. We won't load it.
02/20/2020 03:46:33 - INFO - transformers.tokenization_utils -   loading file /content/model/vocab.json
02/20/2020 03:46:33 - INFO - transformers.tokenization_utils -   loading file /content/model/merges.txt
02/20/2020 03:46:33 - INFO - transformers.tokenization_utils -   loading file None
02/20/2020 03:46:33 - INFO - transformers.tokenization_utils -   loading file /content/model/special_tokens_map.json
02/20/2020 03:46:33 - INFO - transformers.tokenization_utils -   loading file /content/model/tokenizer_config.json
02/20/2020 03:46:33 - INFO - transformers.configu

# TODO: Instructions to give students
* Play around with the presidential speech language model a bit. Compare its outputs to the statistical language model and the LSTM.
* Create a dataset of your own. Find some text online and extract it into a .txt file. Ideally your dataset should contain between 10 and 50 MB of text.
* Break your text file into 2 text files, one for training and one for testing. (We'll need to provide instructions on how to do this)
* Report perplexity of your train set and your test set (we'll need to explain what perplexity is) on GPT-2 without finetuning and GPT-2 with finetuning. Perplexity should be lower after finetuning or something has gone wrong.

# TODO: Analysis questions we could ask students
* Generate 100 unconditioned samples from your finetuned model and 100 unconditioned examples from GPT-without finetuning. Do the samples from your finetuned model have higher word overlap with your test set.
* What happens when you set top-k to 1 vs setting it to some large number?